Bunch of code to inspect each ROI's signal compared to a few basic behavioural variables

In [21]:
from vgatPAG.database.db_tables import *
from vgatPAG.analysis.utils import get_mouse_session_data

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
from random import choices
# import statsmodles.api as sm

from brainrender.colors import colorMap

from fcutils.plotting.plot_elements import plot_shaded_withline
from fcutils.plotting.plot_distributions import plot_kde
from fcutils.plotting.colors import *
from fcutils.plotting.utils import save_figure, clean_axes

# Fetching data

Fetching data involves two steps:
1) Get the data for a specific session (experiment) of a specific mouse from the datajoint database
2) Concatenate akk the different data streams from each recording wihtin the session


In [2]:

# Get all mice
mice = Mouse.fetch("mouse")

# Get all sessions
sessions = {m:(Session & f"mouse='{m}'").fetch("sess_name") for m in mice}

# Get the recordings for each session
recordings = {m:{s:(Recording & f"sess_name='{s}'" & f"mouse='{m}'").fetch(as_dict=True) for s in sessions[m]} for m in mice}



In [3]:
for _mouse in mice:
    print(f'\n{mouse}:')
    for _session in sessions[_mouse]:
        print(f"      |---{_session}")

        recs = Recording().get_sessions_recordings(_mouse, _session)
        print("      |       |--", end='')
        print(*recs, sep=" || ")        


NameError: name 'mouse' is not defined

# Plotting
## Position

Look at the activity of different rois as a function of location in the arena

In [ ]:
# Utility funcs
def get_binned_pc(dflist):
    # Given a list of dataframes with activity binned along one axis,
    # computes PCA and returns the results

    # Get bins
    x_bins = dflist[0].index.values
    int_values = np.array([(i.left + i.right)/2 for i in x_bins])

    # Get signal and remove nans
    binned_signals = np.vstack([df.interpolate().sig.values for df in dflist]).T
    binned_signals[np.isnan(binned_signals)]  = 0

    # Do PCA
    pca = PCA(n_components=1)
    PC = pca.fit_transform(binned_signals)

    # Nan first bins
    PC[(int_values < 250)|(int_values>950)] = np.nan
    
    return int_values, PC


In [ ]:
plt.ioff()

# Params
xbins, ybins = 25, 15
exploration_only = False

# Figure making
for mouse in tqdm(mice):

    all_pcas = []
    for sess in sessions[mouse]:
        x_binned_activity = []
        if exploration_only:
            first_stim = Recording().get_session_first_stim(mouse, sess) 
        else:
            first_stim = True

        if not first_stim: 
            continue

        tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess)

        if len(tracking) != len(signals[0]):
            raise ValueError

        f, axarr = plt.subplots(ncols = 6, nrows=6, figsize=(25, 18))
        axarr = axarr.flatten()
        f.suptitle(f"{mouse} - {sess}")

        for n in np.arange(_nrois):
            # Plot activity projected ont X and Y axis
            trace = pd.DataFrame(dict(
                x = np.int64(tracking['x'].values),
                y = np.int64(tracking['y'].values),
                sig = signals[n],
                isrec = is_rec,
            ))

            if exploration_only: # Keep only up to first stim
                trace = trace[:first_stim]

            if not len(trace): continue

            # ! keep only data for when we are recording
            trace = trace.loc[trace.isrec > 0]
            if not len(trace): continue

            # Plot tracking and binned activity
            axarr[n].set(title=f'ROI {n}')
            axarr[n].hexbin(trace['x'], trace['y'], C=trace.sig, gridsize=xbins, reduce_C_function=np.mean)


            # Bind data along x and y directions
            df = trace.groupby(pd.cut(trace['x'], bins=xbins)).mean()
            int_values = np.array([(i.left + i.right)/2 for i in df.index.values])
            df[int_values < 250] = np.nan
            df[int_values > 950] = np.nan
            plot_shaded_withline(axarr[n], int_values, ((df['sig']/df['sig'].max())*100), z=0, lw=1, color='salmon') # scale the plot to fit nicely
            x_binned_activity.append(df.interpolate())


            df = trace.groupby(pd.cut(trace['y'], bins=ybins)).mean()
            int_values = [(i.left + i.right)/2 for i in df.index.values]
            axarr[n].plot( ((df['sig']/df['sig'].max())*100)+1100, int_values,  lw=4, color='darkseagreen') # scale the plot to fit nicely


            # break
        
        # Plot bin occupancy from tracking
        axarr[n+2].hexbin(tracking['x'], tracking['y'], gridsize=xbins, bins='log', mincnt=1, cmap='inferno')
        _ = axarr[n+2].set(title='Bin occupancy log scale')

        # Plot PCA of activity binned on X axis
        if len(x_binned_activity):
            x_bins, PC = get_binned_pc(x_binned_activity)
            axarr[n+1].plot(x_bins, PC, lw=2, color=darkseagreen) 
            axarr[n+1].set(title='PCA on pop activity binned by X position', xlabel='X position', ylabel='PC1', xlim=[100, 1200])
            all_pcas.append(PC.ravel())

        # Clean axes and save figure
        clean_axes(f)
        f.tight_layout()
        save_figure(f, f'C:\\Users\\Federico\\Documents\\GitHub\\vgatPAG\\plots\\{mouse}_{sess}_ontracking_{"exp" if exploration_only else "all"}', verbose=False)
        plt.close(f)

        # break
    # break

plt.ion()

print('\ndone')

Plot activity of each ROI binned by X position

In [ ]:
# f, axarr = plt.subplots(ncols =  3, nrows=4, figsize=(25, 18))
f, ax = plt.subplots(figsize=(12, 10))
normalize = True
th = 5

px_to_cm = 13.9

endpoints = [[] for i in np.arange(8)]
for n, mouse in tqdm(enumerate(mice)):
    for s, sess in enumerate(sessions[mouse]):

        # axarr[n, s].set(title=f"{mouse} - {sess}")


        tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess)
        # remove speed outliers
        speed[speed>np.nanpercentile(speed, 99)] = np.nanpercentile(speed, 99)



        for nroi in np.arange(_nrois):
            trace = pd.DataFrame(dict(
                    x = np.int64(tracking['x'].values),
                    y = np.int64(tracking['y'].values),
                    s =speed,
                    sig = signals[nroi],
                    isrec = is_rec,
                )).interpolate()

            trace = trace.loc[trace.isrec > 0]

            df = trace.groupby(pd.cut(trace['x'], bins= np.linspace(200, 1100, 9))).mean()
            int_values = np.array([(i.left + i.right)/2 for i in df.index.values])


            y = df.sig

            if normalize:
                y -= y[0]

            if not np.any(np.isnan(y[-1])):
                color = colorMap(y[-1], 'bwr', vmin=-100, vmax=100)
            else:
                color = colorMap(y[-2], 'bwr', vmin=-100, vmax=100)
            ax.plot(int_values, y, 'o-', lw=2, ms=8, mec='k', label=f'roi {nroi}', color=color)
            # axarr[n, s]

            for k, yy in enumerate(y):
                if  not np.any(np.isnan(yy)):
                    endpoints[k].append(yy)


        # axarr[n, s].legend()
        # break
    # break
for k, endp in enumerate(endpoints):
    if k == 0: continue
    plot_kde(data=endp, vertical=True, normto=50, z=int_values[k]+5, ax=ax,  color='gray', zorder=99)


ax.set(title='All ROIs activity binned by speed', xticks=int_values, xticklabels=int_values, xlabel='X position',
                ylabel='Activity', ylim=[-200, 200])

clean_axes(f)
save_figure(f, f'C:\\Users\\Federico\\Documents\\GitHub\\vgatPAG\\plots\\allrois_space_modulation', verbose=False)
plt.show()


# Plotting

## Speed
Plotting CA activity binned by speed

In [ ]:
# Plotting linear regression betwen speed and calcium activity
f, axarr = plt.subplots(ncols =  3, nrows=4, figsize=(25, 18))



for n, mouse in tqdm(enumerate(mice)):
    for s, sess in enumerate(sessions[mouse]):

        axarr[n, s].set(title=f"{mouse} - {sess}")


        tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess)
        # remove speed outliers
        speed[speed>np.nanpercentile(speed, 99)] = np.nanpercentile(speed, 99)



        for nroi in np.arange(_nrois):
            trace = pd.DataFrame(dict(
                    x = np.int64(tracking['x'].values),
                    y = np.int64(tracking['y'].values),
                    s =speed,
                    sig = signals[nroi],
                    isrec = is_rec,
                )).interpolate()

            trace = trace.loc[trace.isrec > 0]


            df = trace.groupby(pd.cut(trace['s'], bins=14)).mean()
            int_values = [(i.left + i.right)/2 for i in df.index.values]
            
            axarr[n, s].plot(int_values, df.sig, 'o-', lw=1, ms=3, label=f'roi {nroi}')

        # axarr[n, s].legend()
        # break
    # break
plt.show()


Plot speed and tracking overlay

In [ ]:

# Params
xbins, ybins = 25, 15
exploration_only = False

f, axarr = plt.subplots(ncols = 3, nrows=4, figsize=(16, 9))
f.suptitle(f"speed and tracking")

# Figure making
for m, mouse in tqdm(enumerate(mice)):
        for s, sess in enumerate(sessions[mouse]):
            tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess)
            # remove speed outliers
            speed[speed>np.nanpercentile(speed, 99)] = np.nanpercentile(speed, 99)
            trace = pd.DataFrame(dict(
                x = np.int64(tracking['x'].values),
                y = np.int64(tracking['y'].values),
                speed = speed,
            ))

        
            # Plot tracking and binned activity
            axarr[m, s].hexbin(trace['x'], trace['y'], C=trace.speed, gridsize=xbins, reduce_C_function=np.mean)
            axarr[m, s].set(title=f'{mouse} - {sess}')

# Clean axes and save figure
clean_axes(f)
# f.tight_layout()
plt.show()

Bin frames in 5 groups based on speed thresholds and look at average ROI activity per Bin

In [ ]:
# f, axarr = plt.subplots(ncols =  3, nrows=4, figsize=(25, 18))
f, ax = plt.subplots(figsize=(12, 10))
normalize = True
th = 5

px_to_cm = 13.9

endpoints = [[], [], [], [], []]
for n, mouse in tqdm(enumerate(mice)):
    for s, sess in enumerate(sessions[mouse]):

        # axarr[n, s].set(title=f"{mouse} - {sess}")


        tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess)
        # remove speed outliers
        speed[speed>np.nanpercentile(speed, 99)] = np.nanpercentile(speed, 99)



        for nroi in np.arange(_nrois):
            trace = pd.DataFrame(dict(
                    x = np.int64(tracking['x'].values),
                    y = np.int64(tracking['y'].values),
                    s =speed,
                    sig = signals[nroi],
                    isrec = is_rec,
                )).interpolate()

            trace = trace.loc[trace.isrec > 0]

            stationary = trace.loc[trace.s < .1]
            slow = trace.loc[(trace.s > .1)&(trace.s < 1)]
            fast = trace.loc[trace.s > th] # trace.s.max()-1

            medium1 = trace.loc[(trace.s > 1)&(trace.s < th/2)]
            medium2 = trace.loc[(trace.s > th/2)&(trace.s < th)]


            y = np.array([stationary.sig.mean(), slow.sig.mean(), medium1.sig.mean(), medium2.sig.mean(), fast.sig.mean()])

            if normalize:
                y -= y[0]

            if not np.any(np.isnan(y[-1])):
                color = colorMap(y[-1], 'bwr', vmin=-100, vmax=100)
            else:
                color = colorMap(y[-2], 'bwr', vmin=-100, vmax=100)
            ax.plot([-1, 0, 1, 2, 3], y, 'o-', lw=2, ms=8, mec='k', label=f'roi {nroi}', color=color)
            # axarr[n, s]

            for k, yy in enumerate(y):
                if  not np.any(np.isnan(yy)):
                    endpoints[k].append(yy)


        # axarr[n, s].legend()
        # break
    # break
for k, endp in enumerate(endpoints):
    if k == 0: continue
    plot_kde(data=endp, vertical=True, normto=.4, z=k+.05-1, ax=ax,  color='gray', zorder=99)

cm_th = th*px_to_cm
ax.set(title='All ROIs activity binned by speed', xticks=[-1, 0, 1 ,2, 3], xticklabels=['stationary', '< 1 cm/frame', f'1<s<{round(cm_th/2, 2)}',
                f'{round(cm_th/2, 2)}<s<{cm_th}', f'> {cm_th} cm/frm'], xlabel='speed')

clean_axes(f)
save_figure(f, f'C:\\Users\\Federico\\Documents\\GitHub\\vgatPAG\\plots\\allrois_speed_modulation', verbose=False)
plt.show()


Do the same but splitting the data based on X position

In [ ]:
# f, axarr = plt.subplots(ncols =  3, nrows=4, figsize=(25, 18))
f, axarr = plt.subplots(nrows=2, figsize=(12, 10))
normalize = True

th = 5
xth = 550


px_to_cm = 13.9

for i in [0, 1]:
    ax = axarr[i]

    
    endpoints = [[], [], [], [], []]
    for n, mouse in tqdm(enumerate(mice)):
        for s, sess in enumerate(sessions[mouse]):

            # axarr[n, s].set(title=f"{mouse} - {sess}")


            tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess)
            # remove speed outliers
            speed[speed>np.nanpercentile(speed, 99)] = np.nanpercentile(speed, 99)



            for nroi in np.arange(_nrois):
                trace = pd.DataFrame(dict(
                        x = np.int64(tracking['x'].values),
                        y = np.int64(tracking['y'].values),
                        s =speed,
                        sig = signals[nroi],
                        isrec = is_rec,
                    )).interpolate()

                trace = trace.loc[trace.isrec > 0]

                if i == 0:
                    trace = trace.loc[trace.x < xth]
                else:
                    trace = trace.loc[trace.x > xth]

                stationary = trace.loc[trace.s < .1]
                slow = trace.loc[(trace.s > .1)&(trace.s < 1)]
                fast = trace.loc[trace.s > th] # trace.s.max()-1

                medium1 = trace.loc[(trace.s > 1)&(trace.s < th/2)]
                medium2 = trace.loc[(trace.s > th/2)&(trace.s < th)]


                y = np.array([stationary.sig.mean(), slow.sig.mean(), medium1.sig.mean(), medium2.sig.mean(), fast.sig.mean()])

                if normalize:
                    y -= y[0]

                if not np.any(np.isnan(y[-1])):
                    color = colorMap(y[-1], 'bwr', vmin=-100, vmax=100)
                else:
                    color = colorMap(y[-2], 'bwr', vmin=-100, vmax=100)
                ax.plot([-1, 0, 1, 2, 3], y, 'o-', lw=2, ms=8, mec='k', label=f'roi {nroi}', color=color)
                # axarr[n, s]

                for k, yy in enumerate(y):
                    if  not np.any(np.isnan(yy)):
                        endpoints[k].append(yy)


            # axarr[n, s].legend()
            # break
        # break
    for k, endp in enumerate(endpoints):
        if k == 0: continue
        plot_kde(data=endp, vertical=True, normto=.4, z=k+.05-1, ax=ax,  color='gray', zorder=99)

    cm_th = th*px_to_cm
    ax.set(title=f"All ROIs activity binned by speed {'left' if i == 0 else 'right'} half of arena", 
                        xticks=[-1, 0, 1 ,2, 3], xticklabels=['stationary', '< 1 cm/frame', f'1<s<{round(cm_th/2, 2)}',
                    f'{round(cm_th/2, 2)}<s<{cm_th}', f'> {cm_th} cm/frm'], xlabel='speed')

clean_axes(f)
save_figure(f, f'C:\\Users\\Federico\\Documents\\GitHub\\vgatPAG\\plots\\allrois_speed_xpos_modulation', verbose=False)
plt.show()


## Plotting tuning to angular velocity

In [ ]:
# f, axarr = plt.subplots(ncols =  3, nrows=4, figsize=(25, 18))
f, ax = plt.subplots(figsize=(9, 6))
normalize = True

th = 0.25

endpoints = [[], [], []]
for n, mouse in tqdm(enumerate(mice)):
    for s, sess in enumerate(sessions[mouse]):

        tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess)
    
        # Remove outliers
        ang_vel[ang_vel>np.nanpercentile(ang_vel, 99)] = np.nanpercentile(ang_vel, 99)
        ang_vel[ang_vel<np.nanpercentile(ang_vel, 1)] = np.nanpercentile(ang_vel, 1)



        for nroi in np.arange(_nrois):
            trace = pd.DataFrame(dict(
                    x = np.int64(tracking['x'].values),
                    y = np.int64(tracking['y'].values),
                    s =speed,
                    avel = ang_vel,
                    sig = signals[nroi],
                    isrec = is_rec,
                )).interpolate()

            trace = trace.loc[trace.isrec > 0]

            stationary = trace.loc[(trace.avel > -th)&(trace.avel<th)]
            cw = trace.loc[trace.avel > th]
            ccw = trace.loc[trace.avel < -th]
            


            y = np.array([ccw.sig.mean(), stationary.sig.mean(), cw.sig.mean()])

            if normalize:
                y -= y[1]

            if np.abs(np.nanmin(y)) > np.nanmax(y):
                color = colorMap(np.nanmin(y), 'bwr', vmin=-20, vmax=20)
                # color = 'gray'
                alpha = 1
            else:
                alpha = 1
                color = colorMap(np.nanmax(y), 'bwr', vmin=-20, vmax=20)

            # if not np.any(np.isnan(y[-1])):
            #     color = colorMap(y[-1], 'bwr', vmin=-100, vmax=100)
            # else:
            #     color = colorMap(y[-2], 'bwr', vmin=-100, vmax=100)
            ax.plot([-1, 0, 1], y, 'o-', lw=2, ms=8, mec='k', label=f'roi {nroi}', color=color, alpha=alpha)
            # axarr[n, s]

            for k, yy in enumerate(y):
                if not np.any(np.isnan(yy)):
                    endpoints[k].append(yy)



for k, endp in enumerate(endpoints):
    if k == 1: continue
    if k == 0:
        normto, z = -.4, -1.1
    elif k == 2:
        normto, z = .4, 1.1
    plot_kde(data=endp, vertical=True, normto=normto, z=z, ax=ax,  color='gray', zorder=99)

ax.set(title='All ROIs activity binned by speed', xticks=[-1, 0, 1], xticklabels=[f'< -{th} deg/frm', f'{-th}<avel<{th} deg/frm', f'> {th} deg/frm'],                          xlabel='angular velocity', ylabel='avg activity')

clean_axes(f)
save_figure(f, f'C:\\Users\\Federico\\Documents\\GitHub\\vgatPAG\\plots\\allrois_avel_modulation', verbose=False)
plt.show()


plot angular velocity over tracking

In [ ]:

# Params
xbins, ybins = 25, 15
exploration_only = False

f, axarr = plt.subplots(ncols = 3, nrows=4, figsize=(16, 9))
f.suptitle(f"speed and tracking")

# Figure making
for m, mouse in tqdm(enumerate(mice)):
        for s, sess in enumerate(sessions[mouse]):
            tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess)
            # Remove outliers
            ang_vel[ang_vel>np.nanpercentile(ang_vel, 99)] = np.nanpercentile(ang_vel, 99)
            ang_vel[ang_vel<np.nanpercentile(ang_vel, 1)] = np.nanpercentile(ang_vel, 1)

            trace = pd.DataFrame(dict(
                x = np.int64(tracking['x'].values),
                y = np.int64(tracking['y'].values),
                ang_vel = ang_vel,
            ))

        
            # Plot tracking and binned activity
            axarr[m, s].hexbin(trace['x'], trace['y'], C=trace.ang_vel, gridsize=xbins, reduce_C_function=np.mean)
            axarr[m, s].set(title=f'{mouse} - {sess}')

# Clean axes and save figure
clean_axes(f)
# f.tight_layout()
plt.show()

# Fitting modelz
Fitting models to predict calcium activity

## Predict position or speed by population activity with naive bayes

In [4]:
from naiveBayes import NaiveBayes

settings: screenshots are set to have transparent backgroundset the parameter "SCREENSHOT_TRANSPARENT_BACKGROUND" to false if you would prefer a not transparent background


In [15]:
N_bins = 25
variable = 'speed' 

results = {}
for m, mouse in tqdm(enumerate(mice)):
        for s, sess in enumerate(sessions[mouse]):
            tracking, ang_vel, speed, shelter_distance, signals, _nrois, is_rec = get_mouse_session_data(mouse, sess, sessions)

               # Prep data
            data = pd.DataFrame({r:sig for r,sig in enumerate(signals)})
            if variable == 'xpos':
                data['to_predict'] = tracking.x
            elif variable == 'speed':
                data['to_predict'] = tracking.s
            else:
                raise ValueError

            # Keep only timepoint during recording
            data = data[is_rec > 0].reset_index(drop=True)
            data = data[:-20] # drop the last few rows because sometimes there's some null values

            # Create categories based on the X position
            categories = pd.cut(data['to_predict'], bins= np.linspace(data['to_predict'].min()-1, data['to_predict'].max()+1, N_bins+1))
            int_values =  np.array([(i.left + i.right)/2 for i in categories.values])
            data = data.drop('to_predict', axis=1)

            # Fit naive bayes  
            nb = NaiveBayes(data, dict(category=int_values))
            labels, predictions = nb.fit_predict(maxrows=None)
            performance = nb.evaluate()

            print(f'\n predicting {variable} | {mouse} - {sess} - {_nrois} rois - performance: {round(performance, 2)}%'+
                        f'[chance: {round(100/N_bins, 2)}%]')

            results[f'{mouse}-{sess}'] = performance





0it [00:00, ?it/s]Model predicted on test set with 17.866% accuracy

 predicting speed | BF161p1 - 19JUN03 - 16 rois - performance: 17.87%[chance: 4.0%]
1it [00:32, 32.59s/it]Model predicted on test set with 10.986% accuracy

 predicting speed | BF161p1 - 19JUN04 - 10 rois - performance: 10.99%[chance: 4.0%]
Model predicted on test set with 19.298% accuracy

 predicting speed | BF164p1 - 19JUN05 - 19 rois - performance: 19.3%[chance: 4.0%]
2it [00:56, 29.90s/it]Model predicted on test set with 25.124% accuracy

 predicting speed | BF164p1 - 19JUN18 - 11 rois - performance: 25.12%[chance: 4.0%]
Model predicted on test set with 12.304% accuracy

 predicting speed | BF164p2 - 19JUL15 - 9 rois - performance: 12.3%[chance: 4.0%]
Model predicted on test set with 15.829% accuracy

 predicting speed | BF164p2 - 19JUN24 - 12 rois - performance: 15.83%[chance: 4.0%]
3it [01:54, 38.26s/it]Model predicted on test set with 21.685% accuracy

 predicting speed | BF164p2 - 19JUN26 - 15 rois - performa

In [25]:
f, ax = plt.subplots(figsize=(12, 9))


ax.bar(np.arange(len(results.keys())), results.values(), color=[.2, .2, .2])
ax.axhline(100/N_bins, lw=2, color='r', ls='--')

ax.set(title=f'Naive Bayes prediction of {variable} from calcium activity', ylabel='Accuracy %', xlabel='Session',
            xticks = np.arange(len(results.keys())), xtickangle=90, 
            xticklabels=list(results.keys()))

clean_axes(f)
save_figure(f, f'C:\\Users\\Federico\\Documents\\GitHub\\vgatPAG\\plots\\naivebayes_{variable}', verbose=False)
plt.show()




AttributeError: Unknown property xtickangle